https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

In [3]:
from langchain.schema import Document
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma
import feedparser

KEYWORD = "RAG"
# ArXiv API URL
url = f'http://export.arxiv.org/api/query?search_query=abs:{KEYWORD}&start=0&max_results=10&sortBy=lastUpdatedDate&sortOrder=descending'
# Parse the API response
feed = feedparser.parse(url)
data = feed.entries

docs = []
for paper in data:
    title = paper["title"]
    abstract = paper["summary"]
    link = paper["link"]
    paper_content = f"Title: {title}\nAbstract: {abstract}"
    paper_content = paper_content.lower()

    docs.append(Document(page_content=paper_content,
                            metadata={"link": link}))
    
embedding_model = CohereEmbeddings(model="embed-english-light-v3.0")

vector_db = Chroma.from_documents(
    documents=docs, embedding=embedding_model)

vector_db.add_documents(docs)

retriever = vector_db.as_retriever()

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {context}"),
    ("human", "{input}"),
])

In [5]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You're an assistant who's good at {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [6]:
from langchain_cohere import ChatCohere
llm = ChatCohere(model="command-r-plus-08-2024",
           max_tokens=256,
           temperature=0.5
           )

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser

# NOTE: default input vars must be 'context' and 'input' for the prompt
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [10]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

'Task decomposition is a concept in computer science and artificial intelligence, particularly relevant to the field of retrieval-augmented generation (RAG) and large language models (LLMs). It is a technique used to break down complex tasks into smaller, more manageable sub-tasks or steps.\n\nIn the context of the research paper you provided, titled "pike-rag: specialized knowledge and rationale augmented generation," task decomposition is specifically mentioned as "knowledge-aware task decomposition." This technique is proposed to address the challenges of extracting and applying specialized knowledge from diverse industrial applications.\n\nKnowledge-aware task decomposition involves the following steps:\n1. Identifying the main task or problem that needs to be solved.\n2. Analyzing the task to understand its complexity and the knowledge requirements.\n3. Decomposing the main task into a set of smaller, more focused sub-tasks or steps. Each sub-task should be associated with specifi

In [15]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

system_prompt_to_reformulate_input = (
    "Given a chat history and the latest user question "
    "which might reference the context in the chat history "
    "formulate a standalone question which can be understood "
    "without chat history. Do NOT answer the question, "
    "just reformulate it is needed and otherwise return it as is."
)

prompt_to_reformulate_input = ChatPromptTemplate.from_messages([
    ("system", system_prompt_to_reformulate_input),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

history_aware_retriever_chain = create_history_aware_retriever(
    llm, retriever, prompt_to_reformulate_input
)

In [ ]:
system_prompt = ("You are an AI assistant called 'ArXiv Assist' that has a conversation with the user and answers to questions based on the provided research papers. "
                 "There could be cases when user does not ask a question, but it is just a statement. In such cases, do not use knowledge from the papers. Just reply back normally and accordingly to have a good conversation (e.g. 'You're welcome' if the input is 'Thanks'). "
                 "If no relevant information is found in the papers, respond with: 'Sorry, I do not have much information related to this. "
                 "If you reference a paper, provide a hyperlink to it. "
                 "Be polite, friendly, and format your response well (e.g., use bullet points, bold text, etc.). "
                 "Below are relevant excerpts from the research papers:\n{context}\n\n"
                 )

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)